### 4) Self Attntion

- CNN as a weighted relation network (탄생배경):
    - CNN과 RN의 단점: 긴 거리의 dependency에 대해서 capture하기 위해서는 깊은 layer가 필요  / RN은 너무 많이 본다. $ \to $ RN과 CNN을 Combine and Generalize 할 수 없을까? 

    - Original (RN) : $ h_{t} = f(x_{t},x_{t-k}) + \cdots + f(x_{t},x_{t}) + f(x_{t},x_{t+k}) $
    - Weighted (CNN): $ h_{t} = \sum_{t'=1}^{T} \mathbb{I}(\left| t' - t \leq k \right|) f(x_{t},x_{t'}) $ 
        - where $ \mathbb{I}(S) = 1 $ if S is true, and 0, otherwise.
    - 각 관계앞에 weighting factor($ \sum_{t'=1}^{T} \mathbb{I}(\left| t' - t \leq k \right|) $)  가 있는 것 처럼 CNN을 RN으로 표현할 수 있다.
    - 이 weight을 0,1 과 같은 constant로 주지 않고, NN이 계산할 수 있도록 하자. 그러면 멀리 있어도 중요하면 무시하지 않도록 하자! $ \to $ 그래서 나온 것이 Self Attention이 다.
    
    
-  T번째 토큰의 Representation을 뽑을 때, t번째 토큰과 모든 다른 토큰을 볼껀데 (RN처럼) 그 관계 중요한지 안중요한지는 $ \alpha $가 결정하도록.
    - $ h_{t} = \sum_{t'=1}^{T} \alpha(x_{t},x_{t'}) f(x_{t},x_{t'}) $ 
    
    
- The weighting function could be yet another neural network. (그냥 NN subgraph (RN) 하나가 더 생긴것으로 볼 수 있다.)
    - Just another subgraph in a DAG: easy to use!: (Weight의 constraint를 주어 0과 1사이로 나오게 할 수 있고)
        - $ \alpha(x_{t},x_{t'}) = \sigma(RN(x_{t},x_{t'})) \in \left[ 0,1 \right] $
    - Perhaps we want to normalize them so that the weights sum to one: (더해서 1이 되도록 하든지 할 수 있다.)
        - $ \alpha(x_{t},x_{t'}) = {{exp(\beta(x_{t},x_{t'}))} \over {\sum_{t''=1}^{T}exp(\beta(x_{t},x_{t'}))}} $, where $ \beta(x_{t},x_{t'})) = RN(x_{t},x_{t'}) $
    - 장점:
        - Able to capture long-range dependencies within a single layer. (CNN에 비해)
        - Able to ignore irrelevant long-range dependencies. (RN에 비해)
    - Graphical한 과정    
        - 모든 Possible pair에 대해 weight을 계산 (Weighting Function $ \alpha $)
        - 그리고 그 둘의 Interection을 계산 (RN을 쓰겠죠.)
        - 그 두개 를 곱해서 (RN은 interaction, Weight은 그것의 가중치)
        - 다 더하여 나온다.    
<img  src="./image/img_SA.PNG" width="60%">
    - Further generalization via multi-head and multi-hop attention.
        - multi-head attention: 아래 Head라는 부분을 늘려 multi-head로 만들어, Long 과 short을 따로 수행하도록 할수 있다
        - multi-hop attention: 계산된 결과를 다시 input으로 넣어(위 그림의 위 벡터) weighting 을 바꿔주는 등의 generalization 방법이 있다. (이런 것이 end2end memory network, Attention is all you need 에서 쭉 나왔으며 multi-head and multi-hop attention은 약간 art로 가는 느낌이다(?!).)
<img  src="./image/img_SA_head.PNG" width="30%">
   - 단점:
      - Quadratic computational complexity(forward path만 보면) $ O(T^{2}) $
      - Some operations cannot be done easily: e.g., counting, (counting같은게 쉽지 않다. 문장, 단어간의 거리와 같은 것)

### 5) RNN

- Online compression of a sequence $ O(T) $
    - self attention은 compress를 안하고 매번 interaction을 보는 것에 반해 sentence를 linear한 타임에 online으로 compress 할 수 있다.
    - $ h_{t} = RNN(h_{t-1},x_{t}) $, where $ h_{0} = 0 $
        - Memory $ h_{t} $ allows it to be Turing complete 

- 과정 
    - 매번 한 토큰을 읽을 때마다 $ h_{t} $ 라는 메모리를 업데이트를 하고 그 토큰을 버리고 다음 토큰으로 가고 또 업데이트 하고 그러면 메모리에 지금까지 읽은 전체 토큰의 정보가 들어간다.
    - 문장 하나를 읽을 때 마지막에 나온 $ h_{T} $ 전체 문장을 summarize해주는 애가 된다.
    - compress를 하다보니 벡터사이즈는 한정되어 있고 length가 길 수록 compress가 어렵고 information loss가 생긴다. 이를 overcome 하기 위해 bidirectional RNN을 쓰기도한다.(양방향으로 읽어 합쳐 t번째 token의 representation을 만든다.)

<img  src="./image/img_bidirectRNN.PNG" width="30%">
- 단점
    - 구조가 Sequential하게 하나씩 읽는 구조다 보니 Modern hardware랑 어긋난다. (동시에 계산이 안되는 parallelize하는게 trivial 한 점으로 인한 속도 저하)
    - 그 외의 training할 때 여러 문제들이 있는데 LSTM과 GRU와 같은 걸 사용하면 대부분 없어진다.
        - Nvdia에서 커널로 만들 놓았고, major framework 에서 노드로 많이 들어가 있으니 LSTM ,GRU 갔다 쓰자

### Summary of sentence  represention

- 지금까지 5가지 방법을 보았다. CBoW, RN, CNN, Self Attention, RNN.

- CBoW을 제외한 4개는 위치를 고려한 방법임을 알 수 있고, 이를 combine해서 사용할 수 있다.

    - ex 1) Google의 translate team에서 나온 paper는 self attention과 RNN을 합쳐 RNN으로 processing을 하고 Self attention을 하니 translate 성능이 올랐다

    - ex 2) 작년의 자신의 PhD student의 논문 character level translation을 할 때는 efficiency를 위해 CCN을 먼저 쓰고 RNN을 쓰니 잘되더라.

- 어차피 모두 differentiable하고 continuous한 노드이므로 어떻게 합치느냐는 어떤 문제를 푸느냐에 따라 특정 combination이 좋고 안좋을 수 있다. 

- Classification을 하기 위함이다 하면 궁극적으로 대부분 averaging을 하거니 RNN을 set reprentaitonn 위에 돌릴 것이다.


- We learned in this lecture…
    - 1) Token Representation:
        - How do we represent a discrete token in a neural network?
        - Training this neural network leads to so-called continuous word embedding.
        - Word2vec, FastText, GloVe (NN을 통해 training한 것들)
    - 2)  Sentence representation (Token Embeddiing이 주어졌을 때, 이런 것들을 어떻게 잘 합쳐서 Sentence representation로 만드는지. 대부분 여기를 중점적으로 강의한 이유는 이를 기반으로 문장을 어떻게 generate하고, 좋은 애를 선택하는지 등 다양한 것을 볼 것이라서 이다.)
        - How do we extract useful representation from a sentence?
        - We learned five different ways to do so: CBoW, RN, CNN, Self-Attention, RNN
        
        


        
### Q & A

- Q1. polysemy problem을 어떻게 disambiguate하는가.
    - A: Context를 보면 disambiguate이 되는데, 이 중에 word sense disambiguate 것들이 있다. 2가지를 생각해 봐야하는데 각각의 벡터들이 (token을 vector로 represent할 때) 굉장히 high dimensional sapce에 있다. 이는 단어들간의 관계가 (1d에서는 한 점의 neighbor가 2개 2d 에서는 한 4개 정도가 나올수 있는데.) hight dim에서는 굉장히 많은 것들이 neighbor가 되어 여러 의미를 encode를 할 수 있다.
    - 이제 이 중에 무엇을 골라야하는가 하는 문제가 생기는데 이는 sentence representation에 들어가게 된다.(해결할 수 있다?) 즉 sentence representation한다는 것 자체가 token representation들을 어떻게 합치느냐. 합칠 때 즉 context 주위 단어들 어떻게 받느냐  다시 문제를 푸는데 필요하면은(task oriented 사실 이게 가장 중요한데) 알아서 sentence representation extraction level에서 polysemysms disambiguate해 질 것이다.
        - 내 생각은 Task에 따라 데이터를 잘 설저앟면 다의어는 NN에서 알아서 잘 extract되어 표현될 것이다는 느낌.
        
        
- Q2. Training Set에 없는 단어(모르는 단어)는 어떻게 처리(표현) 하는가.
    - 지금까지 vocab이 주어졌다는 것 가정하에 이루어지긴 했다. 그러면 새로운 단어가 생기는 문제는 먼저 어떤 level에서 token을 define 했냐를 생각해야한다. 토큰 define을 syllable(음절) 또는 alphabet level이었으면 새로운 단어는 가끔 generalize하기도 한다.(새 단어는 alphabet embedding을 어떻게 하느냐의 차이이므로 이런 경우에 generalize 하기도 한다.)
    - 단어나 morpheme(형태소) level에서하면은 진짜 새로운 단어가 되는데, 이를 해결하기 위해 여러가지 방법 중 하나는 새로운 단어를 쓰는 example들을 training set에 추가해 fine tuning 하는 것
    - 아니면 새로운 단어 (모르는 단어)에 대해 기존에 존재하는 embedding을 이용해 만들어주는 방법도 있다. (이것이 최근에 나온 MS research paper의 universal lexical representation의 방법: 여기서는 하나의 언어를 다른 하나의 언어로 어떻게 embedding 하냐에 대한 문제를 다룸)
    - 즉 어떤 level에서 token을 정하느냐에 따라 해결 방법이 다를 것이다.    
    
- Q3.  분류 모델 훈련 완료후, 새로운 클래스가 등장했을 때 어떻게 하는지? (챗봇 개발자의 질문)
    - 1) 새로 training을 한다. 
    - 2) 데이터가 너무 적고, weight vector가 하나 늘어난 셈이니 그 하나 늘어나 weight vector만 estimation을 다시 할 수 있다.(나머지는 그대로 놓고)
        - DAG로 관점으로 보는 것 중 하나의 장점이 Gradient를 backpropa할 때 전부 다를 계산하여 update할 필요 없다는 것이다.
        - BUT 그 때 문제는 과연 이게 잘 optimal 해진 것일까.
            - 여러 다른 class의 description이 나올 것인데 이를 통해 network이 알아서 한다(?) 
            - 이런 쪽으로 zero shot, few shot learnig을 보면 (Jaon Weston의 와삽?)을 보면 알 수 있고 실제로 해보면 잘 된다.
            
- Q4. 임베딩에서 “가깝다”는 벡터 공간에서 코사인 유사도를 말하는 건지? 아니면 다른 distance metrics을 정의해서 사용하는지?          

    - 예를 들어 CBoW Classifier를 train했다 해보자. 이게 W2V할 때, 이 Network이 그대로 쓰는 것인데, class가 뭔지 잘 지정하여 Vector가 나오는 것이다.
    - Vector간의 관계가 Token들이 비슷하면, vector들이 비슷하고 token이 다르면 vector가 다른데 그게 어떤 distance metric이냐? 
        - 다양하게 할 수있다. Euclidean, Mahalanobis distance 또는 arbitrary하게 Riemannian Manifold를 define해서 할 수 있는데 어떤 것이 맞는 지는 Word Vector가 나온 후에 어떤 Network 가 쓰여져 씌였는지가 가장 중요하다.
        - 만약 Average 단계에서 각 벡터에 대해 length를 Normalize하고 average를 했으면 Cosine distance가 맞는 metric이다.(magnitude를 무시해야하고 Angle만 보는 것이 좋으니까)
        - BUT 사실 Tomas Mikolov가 Cosine Similarity가 2012년에 좋다고 하니까 그런가보다 한 것이지.
        - 어떤것이 최적인가는 어떻게 train을 하는지와 어떤 Downstream Task(Clustering, Retrieval 등등)하는 지에 따라 다르다.
        - 즉 Distance Metric도 hyperparameter라고 할 수 있다.
    
<img  src="./image/img_Cbowclassifier.PNG" width="60%">    
        
- Q5. Capsule network로 텍스트 분류 문제를 사용하는건 어떤지? (이미지의 Capsule network가 Region을 잘 잡는 다고 하는데...)
    
    - Text에서는 CapsNet이 잘 사용되지 않는다. CapsNet의 목표는 CNN의 Pooling에서 invariance를 찾는 것인데(조금의 perturbation이 있어도 output이 그대로 있어야 한다)는 것을 하는 것인데, Text에서는 중요한 perturbation이 무엇인지 정의가 잘 되어있지 않다.
    - 물론 중간에는 사용할 수 있겠다. sentence representation을 뽑아낸 후에 CapsNet을 할 수 있겠다. 아직 연구를 더 해봐야 한다.
   

- Q6. Attention 모델에서 전체 문장을 보지 않고 고정된 윈도우 사이즈를 정해서 할 수 있을 것 같은데 현업에서의 rule of thumb의 길이는 있는가?
    - 나중에 말할 것 중의 하나였는데, token을 character하나하나 마다 하면 너무 길어지니까 (문장 표현이) 속도 문제가 있으니 이를 어떻게 줄이느냐가 큰 issue인데
    - 보통 사람들 그리고 조경현 교수님이하는 것이 convnet을 통해 local pattern을 capture를 하고 pooling을 통해 줄이는 방법이 있고
    - 구글 nlp, brain팀에서 나온 것은 그냥 ratio를 주어서 각각을 하나의 벡터를 바꾼다든지의 방법 들이 있다.
    - 하지만 이런 것이 어려운게, 줄이려는 이유가 2가지 여서 그렇다. 하나는 성능이 좋아지지 않을까. 그리고 Computation 성능을 높일 수 있지 않을까. 일 것이다. 이것이 trade-off를 찾는 것이어서 정답은 어느 부분에 초점을 두느냐에 따라 다를 것이다.
    - 나중에 Character level translation을 할 때는 어떤 식으로 design choice를 해야하는지 말하겠다.
    
- Q7. Self attention은 어떻게 학습하고, optimize 되는지?    
    - 어떤 문제에 따라 Attention이 다르겠지만, 일반적으로 쓰는 Backpropa와 SGD 쓰는 것이 적용 가능하다. 
    - Optimization이 어려운 면이 없지않아 있다. 설명은 안했지만
    - Backpropa, GD는 여기서 Computation Graph적으로 보면 Credit Assignment process이다.
        - 각 노드 마다  Loss function을 계산하는 노드들과 parameter에 대해, Loss function이 이런 값이 된 것에는 너가 이런식으로 contribute 했어를 전파시켜주는 것이다. 전파된 양을 보고 positive 하게 많이 했으면 값을 올리고 negative하게 했으면 값을 줄이는 것이 BP와 GD가하는 일일 텐데
    - 초반에는 Attention의 weighting factor가 제대로 안되있을 것이다. 이런 상횡에서 Weighting 굉장히 flat하게 나올 것인데 이상황에서 credit assign을 할 수 있느냐의 문제가 생길 수 있다.
    - 다시 말해, Weighting Function 알파가 flat하면 누가 중요한지 몰라 learning이 잘 안된다.
    - 따라서 parameter initialization을 어떻게 하는 것이 까다롭지만, 하지만 최근의 Self Attention을 이용한 Machine Translation을 만들 때 어떤 hyperparameter가 중요한지 나와있는 Technical report를 참고해 해결할 수도 있겠다.
    
    
    
    
    

### Reference: 

- https://www.edwith.org/deepnlp 조경현교수님, 딥러닝을 이용한 자연어 처리 강의 및 강의 자료 